In [10]:
from datahelpers.constants import iden, ye, ai, ps, up, dn, ar, ni, cexp, qcexp, nw, wi, dist, rdist, pm, \
                                    cpop, cden, ct, affs, aus
from os.path import expanduser, join
import pandas as pd
from bm_support.add_features import generate_feature_groups
from bm_support.add_features import normalize_columns
from bm_support.supervised_aux import study_sample, metric_selector
from functools import partial
from multiprocessing import Pool
from copy import deepcopy
import warnings
from numpy.random import RandomState
import gzip
import pickle
import argparse
from sklearn.model_selection import train_test_split
from bm_support.supervised import simple_stratify
from bm_support.supervised import select_features_dict, logit_pvalue, report_metrics
from sklearn.ensemble import RandomForestRegressor

%load_ext autoreload
%autoreload 1
%aimport bm_support.add_features
%aimport bm_support.supervised_aux
%aimport bm_support.supervised

warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
an_version = 12

# origin = 'litgw'
# version = 1

# origin = 'lit'
# version = 8

origin = 'gw'
version = 11

# model_type = 'rf'
# seed0 = 17
n_trials = 1
n_subtrials = 10
n_estimators = 17
# n_estimators = 55
datapath = None
# seed0 = 13
seed0 = 1
n_jobs = 1
verbose = True
model_type = 'lr'
model_type = 'rf'

In [3]:
min_log_alpha = -1
max_log_alpha = 1
log_reg_dict = {'min_log_alpha': min_log_alpha, 'max_log_alpha': max_log_alpha}

eps = 0.2
upper_exp, lower_exp = 1 - eps, eps
# thrs = [-1e-8, lower_exp, upper_exp, 1.0001e0]
if datapath:
    col_families = generate_feature_groups(expanduser(join(datapath, 'v12_columns.txt')))
else:
    col_families = generate_feature_groups(expanduser('~/data/kl/columns/v12_columns.txt'))

if verbose:
    print('Number of col families: {0}. Keys: {1}'.format(len(col_families), sorted(col_families.keys())))

col_families = {k: v for k, v in col_families.items() if 'future' not in k}
if verbose:
    print('Number of col families (excl. future): {0}. Keys: {1}'.format(len(col_families),
                                                                         sorted(col_families.keys())))

columns_interest = [x for sublist in col_families.values() for x in sublist]
if datapath:
    df_path = expanduser(join(datapath, '{0}_{1}_{2}.h5'.format(origin, version, an_version)))
else:
    df_path = expanduser('~/data/kl/final/{0}_{1}_{2}.h5'.format(origin, version, an_version))
df = pd.read_hdf(df_path, key='df')

['cpop', 'cden', 'ksst', 'nhi', 'affiliations_affind', 'authors_affind', 'future_affind', 'past_affind', 'affiliations_suppind', 'authors_suppind', 'future_suppind', 'past_suppind', 'affiliations_comm_size', 'authors_comm_size', 'future_comm_size', 'past_comm_size', 'affiliations_ncomms', 'authors_ncomms', 'future_ncomms', 'past_ncomms', 'affiliations_ncomponents', 'authors_ncomponents', 'future_ncomponents', 'past_ncomponents', 'affiliations_size_ulist', 'authors_size_ulist', 'future_size_ulist', 'past_size_ulist', 'pre_authors', 'pre_affs']
{'cpop': 8, 'cden': 8, 'ksst': 8, 'nhi': 2, 'affiliations_affind': 4, 'authors_affind': 4, 'future_affind': 4, 'past_affind': 4, 'affiliations_suppind': 4, 'authors_suppind': 4, 'future_suppind': 4, 'past_suppind': 4, 'affiliations_comm_size': 4, 'authors_comm_size': 4, 'future_comm_size': 4, 'past_comm_size': 4, 'affiliations_ncomms': 4, 'authors_ncomms': 4, 'future_ncomms': 4, 'past_ncomms': 4, 'affiliations_ncomponents': 4, 'authors_ncomponents

In [4]:
# mask: literome - mask out a specific interaction
mask_lit = (df[up] == 7157) & (df[dn] == 1026)

# mask:  interaction with more than 3 claims
thr = 3
mask_len_ = (df.groupby(ni).apply(lambda x: x.shape[0]) > thr)
mask_max_len = df[ni].isin(mask_len_[mask_len_].index)
print(mask_max_len.shape[0], sum(mask_max_len))

# mask : interactions which are between
eps_window_mean = 0.1
mean_col = 0.5
mask_exp = ((df[cexp] <= lower_exp - eps_window_mean) | (df[cexp] >= upper_exp + eps_window_mean)
            # | ((df2[cexp] <= (mean_col + eps_window_mean)) & (df2[cexp] >= (mean_col - eps_window_mean)))
            )

feature_dict = deepcopy(col_families)
# families = ['affiliations_affind', 'affiliations_comm_size', 'affiliations_ncomms',
#             'affiliations_ncomponents', 'affiliations_size_ulist', 'affiliations_suppind',
#             'ai', 'ar', 'authors_affind', 'authors_comm_size', 'authors_ncomms', 'authors_ncomponents',
#             'authors_size_ulist', 'authors_suppind', 'cden', 'citations',
#             'cite_count', 'cpop', 'delta_year', 'ksst', 'lincscomm_size', 'lincssame_comm',
#             'litgwcsize_dn', 'litgwcsize_up',
#             'litgweff_comm_size', 'litgwsame_comm', 'nhi', 'past_affind',
#             'past_comm_size', 'past_ncomms', 'past_ncomponents',
#             'past_size_ulist', 'past_suppind', 'pre_affs', 'pre_authors', 'time']
families = ['affiliations_comm_size',
            'ai', 'ar', 'cden', 'citations',
            'cite_count', 'cpop', 'delta_year', 'ksst', 'lincscomm_size', 'lincssame_comm',
            'litgweff_comm_size', 'litgwsame_comm', 'nhi', 'past_affind',
            'past_comm_size', 'time']

feature_dict = {k: v for k, v in feature_dict.items() if k in families}

trial_features = [x for sublist in feature_dict.values() for x in sublist]

feature_dict_inv = {}
for k, v in feature_dict.items():
    feature_dict_inv.update({x: k for x in v})

# mask: not nulls in trial features
masks = []
for c in trial_features:
    masks.append(df[c].notnull())

mask_notnull = masks[0]
for m in masks[1:]:
    mask_notnull &= m

print('Experimental mask len {0}'.format(sum(mask_exp)))
print('Number of trial features: {0}'.format(len(trial_features)))
print('Number of notnull entries (over all features): {0} from {1}'.format(sum(mask_notnull), mask_notnull.shape))

if origin != 'gw':
    mask_agg = mask_notnull & ~mask_lit
else:
    mask_agg = mask_notnull

dfw = df.loc[mask_agg].copy()

#metric to optimize for
mm = 'accuracy'
# mm = 'precision'

nmax = 5000

rns = RandomState(seed0)
seeds = rns.randint(nmax, size=n_trials)

if model_type == 'lr':
    dfw = normalize_columns(dfw, trial_features)

15494 7878
Experimental mask len 887
Number of trial features: 90
Number of notnull entries (over all features): 14943 from (15494,)


In [5]:
def study_sample_cont(seed, dfw, dist, feature_dict,
                 metric_mode, model_type, n_subtrials, n_estimators,
                 log_reg_dict={'min_log_alpha': -2., 'max_log_alpha': 2.},
                 verbose=False):
    nmax = 10000
    metric_uniform_exponent = 0.5
    mode_scores=None
    min_log_alpha, max_log_alpha = log_reg_dict['min_log_alpha'], log_reg_dict['max_log_alpha']
    feature_dict_inv = {}
    for k, v in feature_dict.items():
        feature_dict_inv.update({x: k for x in v})

    rns = RandomState(seed)
    df_train, df_testgen = train_test_split(dfw, test_size=0.4,
                                            random_state=rns, stratify=dfw[dist])

    df_valid, df_test = train_test_split(df_testgen, test_size=0.5,
                                         random_state=rns)

    vc = df_train[dist].value_counts()
    if verbose and vc.shape[0] < 5:
        print('*** df_train dist vc')
        print(vc)
    # training on the normalized frequencies
    df_train2 = simple_stratify(df_train, dist, seed, ratios=(2, 1, 1))

    if model_type == 'rf':
        param_dict = {'n_estimators': n_estimators, 'max_features': None, 'n_jobs': 1}
    else:
        param_dict = {'n_jobs': 1}

    meta_agg = []
    if model_type == 'rf':
        enums = rns.randint(nmax, size=n_subtrials)
    elif model_type == 'lr':
        delta = (max_log_alpha - min_log_alpha) / n_subtrials
        enums = 1e1 ** np.arange(min_log_alpha, max_log_alpha, delta)
    else:
        enums = []
    return df_train2, df_test, df_valid

In [11]:
df_train, df_test, df_valid = study_sample_cont(dfw=dfw, seed=seed0, dist=dist, feature_dict=feature_dict, metric_mode=mm,
                                               model_type=model_type,
                                               n_subtrials=n_subtrials, n_estimators=n_estimators,
                                               log_reg_dict=log_reg_dict, verbose=verbose)


*** df_train dist vc
1.0    7944
0.0     826
2.0     195
Name: guess, dtype: int64


In [20]:
target_column = 'litgw_comm_im_undir_wei_pNone_eff_comm_size'
feature_dict_new = {k: list(v) for k, v in feature_dict.items() if target_column not in v}
len(feature_dict), len(feature_dict_new)

(17, 16)

In [15]:
if model_type == 'rf':
    param_dict = {'n_estimators': n_estimators, 'max_features': None, 'n_jobs': 1}
else:
    param_dict = {'n_jobs': 1}

meta_agg = []
if model_type == 'rf':
    enums = rns.randint(nmax, size=n_subtrials)
elif model_type == 'lr':
    delta = (max_log_alpha - min_log_alpha) / n_subtrials
    enums = 1e1 ** np.arange(min_log_alpha, max_log_alpha, delta)
else:
    enums = []

In [17]:
ii = enums[0]
if model_type == 'rf':
    param_dict['random_state'] = ii
elif model_type == 'lr':
    param_dict['C'] = ii

In [18]:
param_dict

{'n_estimators': 17, 'max_features': None, 'n_jobs': 1, 'random_state': 3980}

In [22]:
model_dict = param_dict
y_train, y_test = df_train[target_column], df_test[target_column]

chosen_features = []
chosen_metrics = []
chosen_total_metrics = []
feature_dict_dyn = deepcopy(feature_dict)
feature_dict_inv = {}
for k, v in feature_dict.items():
    feature_dict_inv.update({x: k for x in v})

if verbose:
    print('model_dict {0}'.format(model_dict))

model_dict {'n_estimators': 17, 'max_features': None, 'n_jobs': 1, 'random_state': 3980}


In [48]:
max_features_consider = 5
trial_features = [x for sublist in feature_dict_dyn.values() for x in sublist]
len(trial_features)
cur_features = trial_features[:20]

In [26]:
X_train, X_test = df_train[cur_features], df_test[cur_features]

In [38]:
model_ = RandomForestRegressor(**model_dict)

In [39]:
model_.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=17, n_jobs=1,
           oob_score=False, random_state=3980, verbose=0, warm_start=False)

In [43]:
rmetrics = report_metrics(model_, df_valid[cur_features], df_valid[target_column],
                          problem_type='regression')
rmetrics

{'corr': 0.5273485462658444,
 'mse': 0.01312455460048995,
 'main_metric': 0.01312455460048995}

In [45]:
rmetrics = report_metrics(model_, df_test[cur_features], df_test[target_column],
                          problem_type='regression')
rmetrics

{'corr': 0.4673735955828734,
 'mse': 0.013975235251299248,
 'main_metric': 0.013975235251299248}

In [46]:
rmetrics = report_metrics(model_, df_train[cur_features], df_train[target_column],
                          problem_type='regression')
rmetrics

{'corr': 0.7831827946259942,
 'mse': 0.021600409081096797,
 'main_metric': 0.021600409081096797}

In [33]:
print(list(zip(cur_features, model.feature_importances_)))

[('cpop', 0.09261631525873423), ('cpoprc', 0.043488408358661045), ('cpop1', 0.022376554564349057), ('cpoprc1', 0.016011590539879238), ('cpop2', 0.028528700022709363), ('cpoprc2', 0.03959154151798805), ('cpop3', 0.013361119822479244), ('cpoprc3', 0.03111474886704313), ('cden', 0.25957864400978375), ('cdenrc', 0.12507584497973356), ('cden1', 0.013255175142452284), ('cdenrc1', 0.016355940292462137), ('cden2', 0.013748914775664516), ('cdenrc2', 0.040361991787341976), ('cden3', 0.016642873913723117), ('cdenrc3', 0.016240996935663138), ('ksst', 0.10027124110025597), ('ksstrc', 0.10986762449722434), ('ksst1', 0.0), ('ksstrc1', 0.0015117736138518453)]


In [ ]:
r = select_features_dict(df_train2, df_test, target,
                       feature_dict,
                       model_type=model_type,
                       max_features_consider=8,
                       metric_mode=metric_mode,
                       mode_scores=mode_scores,
                       metric_uniform_exponent=metric_uniform_exponent,
                       model_dict=param_dict,
                       verbose=verbose)

In [49]:
while len(chosen_features) <= max_features_consider and feature_dict_dyn:
    trial_features = [x for sublist in feature_dict_dyn.values() for x in sublist]
    scalar_metrics = []
    cur_metrics = []
    for tf in trial_features:
        cur_features = chosen_features + [tf]
        X_train, X_test = df_train[cur_features], df_test[cur_features]
        if model_type == 'rf':
            model = RandomForestClassifier(**model_dict)
        else:
            model = LogisticRegression(**model_dict)
        model.fit(X_train, y_train)
        rmetrics = report_metrics(model, X_test, y_test, mode_scores, metric_uniform_exponent, metric_mode)
        # y_pred = model.predict(X_test)

        # corr = np.corrcoef(y_pred, y_test)[0, 1]
        # accuracy = accuracy_score(y_test, y_pred)
        # precision = precision_score(y_test, y_pred, average=mode_scores)
        # recall = recall_score(y_test, y_pred, average=mode_scores)
        # f1 = f1_score(y_test, y_pred, average=mode_scores)
        # conf_matrix = confusion_matrix(y_test, y_pred)
        # report['confusion'] = conf_matrix
        # report['class_report'] = classification_report(y_test, y_pred)
        # vector_metrics.append((precision, recall, f1))
        # sc_vector_metrics = [np.sum(x**metric_uniform_exponent) for x in [precision, recall, f1]]
        scalar_metrics.append(rmetrics['main_metric'])
        cur_metrics.append(rmetrics)

    add_index = np.nanargmax(np.array(scalar_metrics))

    if len(chosen_metrics) > 0:
        potential_improvement = 1 - chosen_metrics[-1] / scalar_metrics[add_index]
        if potential_improvement < eps_improvement:
            if verbose:
                print('Terminating early: no improvement.')
            break
    else:
        potential_improvement = 1

    current_feature = trial_features[add_index]
    feature_group = feature_dict_inv[current_feature]

    chosen_metrics.append(scalar_metrics[add_index])

    chosen_features.append(current_feature)
    chosen_total_metrics.append(cur_metrics[add_index])

    if verbose:
        print('nf: {0} cfeature: {1} metric: {2:.3f} metric_improv: {3:.2f} %'.format(len(cur_features),
              (current_feature[:27]+'...').ljust(30),
              chosen_metrics[-1], 100*potential_improvement))

    if len(feature_dict[feature_group]) - len(feature_dict_dyn[feature_group]) < 1:
        feature_dict_dyn[feature_group].remove(current_feature)
    else:
        del feature_dict_dyn[feature_group]

NameError: name 'RandomForestClassifier' is not defined

In [35]:
func = partial(study_sample, dfw=dfw, dist=dist, feature_dict=feature_dict, metric_mode=mm,
               model_type=model_type,
               n_subtrials=n_subtrials, n_estimators=n_estimators,
               log_reg_dict=log_reg_dict, verbose=verbose)

print('Starting {0} processes...'.format(n_jobs))

# lr seems to run 4x
if n_jobs > 1:
    with Pool(n_jobs) as p:
        meta_report = p.map(func, seeds)
else:
    meta_report = list(map(func, seeds))

# if datapath:
#     fout = expanduser(join(datapath, '{0}_{1}_{2}_{3}_seed0_{4}.report.pgz'.format(origin, version,
#                                                                                    an_version, model_type, seed0)))
# else:
#     fout = expanduser('~/data/kl/reports/{0}_{1}_{2}_{3}_seed0_{4}.report.pgz'.format(origin, version,
#                                                                                       an_version,
#                                                                                       model_type, seed0))

# with gzip.open(fout, 'wb') as fp:
#     pickle.dump(meta_report, fp)

15494 7878
Experimental mask len 887
Number of trial features: 90
Number of notnull entries (over all features): 14943 from (15494,)
Starting 1 processes...
*** df_train dist vc
1.0    7944
0.0     826
2.0     195
Name: guess, dtype: int64
model_dict {'n_estimators': 17, 'max_features': None, 'n_jobs': 1, 'random_state': 4183}
nf: 1 cfeature: litgw_comm_im_undir_unwei_p... metric: 0.354 metric_improv: 100.00 %
nf: 2 cfeature: delta_year...                  metric: 0.425 metric_improv: 16.61 %
nf: 3 cfeature: litgw_comm_im_undir_wei_pNo... metric: 0.477 metric_improv: 10.88 %
nf: 4 cfeature: past_affind3...                metric: 0.510 metric_improv: 6.58 %
nf: 5 cfeature: yearspan_flag...               metric: 0.512 metric_improv: 0.30 %
nf: 6 cfeature: litgw_comm_im_dir_wei_pNone... metric: 0.538 metric_improv: 4.86 %
nf: 7 cfeature: past_affind2...                metric: 0.557 metric_improv: 3.42 %
Terminating early: no improvement.
model_dict {'n_estimators': 17, 'max_features': Non

In [37]:
meta_report[0][1][0]

{'run_par': 235,
 'current_features': ['litgw_comm_im_undir_unwei_p95_eff_comm_size',
  'delta_year',
  'litgw_comm_im_undir_wei_pNone_eff_comm_size',
  'past_affind3',
  'yearspan_flag',
  'litgw_comm_im_dir_wei_pNone_dyn_same_comm',
  'past_affind2'],
 'current_metrics': [0.3543356575761969,
  0.4248993487140507,
  0.4767629902036166,
  0.5103540113326976,
  0.511879927299675,
  0.5380078880731337,
  0.5570867439071006],
 'current_vector_metrics': [{'corr': 0.19311290312487198,
   'accuracy': {'normal': 0.7229842756774841, 'balanced': 0.3543356575761969},
   'vector': {'precision': array([0.30291262, 0.96029056, 0.05378973]),
    'recall': array([0.57777778, 0.74492863, 0.38596491]),
    'f1': array([0.39745223, 0.83900994, 0.0944206 ])},
   'macro': {'precision': 0.43899763643709816,
    'recall': 0.5695571050507979,
    'f1': 0.44362759101301735},
   'exponent': {'precision': 0.5874150917795194,
    'recall': 0.7481567239939292,
    'f1': 0.61789744577049},
   'main_metric': 0.3543

In [30]:
len(meta_report)

2

In [32]:
meta_report[0][1][1]

10